In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from sotodlib import core
import sotodlib.io.load as io_load

from moby2.analysis import socompat
socompat.register_loaders()

import tools

#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

In [2]:
from sotodlib.core import FlagManager

import sotodlib.flags as flags
import sotodlib.sim_flags as sim_flags

import sotodlib.tod_ops.filters as filters

from sotodlib.tod_ops import fourier_filter, rfft, detrend_data

In [3]:
import tools
import importlib
importlib.reload(tools)
from tools import in_range

In [4]:
nersc_todsims = '/global/project/projectdirs/sobs/todsims/pipe-s0001/v4/context.yaml'

In [5]:
context = core.Context(nersc_todsims)
context
context['tags']
context['obs_colon_tags']

['band', 'wafer', 'fcode']

In [6]:
obs_list = context.obsdb.get()
num = 200
my_obs = obs_list[num]
my_obs
my_obs['obs_id']

'CES-Atacama-LAT-Tier1DEC-035..-045_RA+040..+050-11-1_MFS'

In [7]:
my_obs['obs_id']

'CES-Atacama-LAT-Tier1DEC-035..-045_RA+040..+050-11-1_MFS'

In [8]:
# get detsets from an observation
det_sets = context.obsfiledb.get_detsets(my_obs['obs_id'])
det_sets
det_set = det_sets[0]

# get single detector from one detector set
dets = context.obsfiledb.get_dets(det_set)
#det
type(dets)
dets[0:2]

['26_000_MFS1_A', '26_000_MFS1_B']

In [9]:
tod = context.get_obs(my_obs, dets=dets[5:10])
tsamp = np.median(np.diff(tod.timestamps))
n_cut = int(10//tsamp)
print('Trimming in time...')

tod.restrict('samps', (n_cut, tod.samps.count-n_cut))
print('The TOD now has {} samples'.format(tod.samps.count))
if 'badness' in tod:
        tod.move('badness', None)
sim_flags.add_random_glitches(tod, params={'n_glitch':2, 'sig_n_glitch' : 0,'h_glitch':5}, signal='badness', overwrite='True')

if 'bad_signal' in tod:
        tod.move('bad_signal', None)
# wrap the glitches to the tod
tod.wrap('bad_signal', tod.signal+tod.badness, [(0, tod.dets), (1, tod.samps)])

for i in range(tod.dets.count):
    
    true_glitches = tod.flags.true_glitches[i]
    true_glitches.ranges()
    
    
    found_glitches = flags.get_glitch_flags(tod, signal=signal_name, overwrite=True).ranges[i]
    found_glitches.ranges()
    
    
    results = true_glitches.mask()*found_glitches.mask()
    print('1:', np.sum(true_glitches.mask()))
    print('2:', found_glitches.ranges().size/2)
    print('3:', np.sum(results))

Trimming in time...
The TOD now has 183478 samples


NameError: name 'signal_name' is not defined

In [535]:
def test_glitch(tod):
    tod
    found_glitches = flags.get_glitch_flags(tod, signal=signal_name, overwrite=True).ranges
    true_glitches = tod.flags.true_glitches
    
    found_glitches_ranges2D = np.vstack([r.mask() for r in found_glitches])
    true_glitches_ranges2D = np.vstack(r.mask() for r in true_glitches)
    
    result = found_glitches_ranges2D*true_glitches_ranges2D
    
    
    true_sum = true_glitches_ranges2D.sum()
    found_sum = sum([mr.ranges().size/2 for mr in found_glitches])
    found_true_sum = result.sum()
    
    detection_rate = found_true_sum/true_sum
    true_positive_rate = found_true_sum/found_sum
    false_positive_rate = 1 - true_positive_rate
    
    return detection_rate, true_positive_rate, false_positive_rate